## Imported Necessary Packages and Connection Strings

In [1]:
# All query results are returned as Pandas DataFrames 
import pandas as pd

# Needed for %sql Magic
%load_ext sql

import sqlite3

conn = sqlite3.connect("./CourseDataWarehouseFinal.db")

In [2]:
%sql sqlite:///CourseDataWarehouseFinal.db

'Connected: @CourseDataWarehouseFinal.db'

## Drop Tables if they Exist

In [3]:
%%sql
DROP TABLE IF EXISTS dimCourses;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

In [4]:
%%sql
DROP TABLE IF EXISTS dimCap;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

In [5]:
%%sql
DROP TABLE IF EXISTS dimTerm;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

In [6]:
%%sql
DROP TABLE IF EXISTS dimCRN;

 * sqlite:///CourseDataWarehouseFinal.db
Done.


[]

## Create Tables and Insert Info from Database tables

In [7]:
%%sql

DROP TABLE IF EXISTS dimCourses;

CREATE TABLE dimCourses(
    program_code VarChar NOT NULL,
    catalog_id VarChar NOT NULL,
    course_title TEXT NOT NULL,
    credits INT NOT NULL,
    prereqs VarChar,
    coreqs VarChar,
    fees VarChar,
    attributes TEXT,
    description TEXT,
    PRIMARY KEY (program_code, catalog_id)
);
    
INSERT INTO dimCourses (program_code, catalog_id, course_title, credits, prereqs, coreqs, fees, attributes, description)
SELECT DISTINCT program_code, catalog_id, course_title, credits, prereqs, coreqs, fees, attributes, description
FROM CourseCatalog;


 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
2220 rows affected.


[]

In [8]:
%%sql

DROP TABLE IF EXISTS dimTerm;

CREATE TABLE dimTerm(
    term VarChar NOT NULL,
    title TEXT NOT NULL,
    crn INT NOT NULL,
    credits INT NOT NULL,
    Location VarChar,
    Primary_Instructor TEXT NOT NULL,
    PRIMARY KEY (crn, term)
);

INSERT INTO dimTerm(term, title, crn, credits, Location, primary_instructor)
SELECT DISTINCT term, title, crn, credits, Location, primary_instructor
FROM courses_final;

 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
13860 rows affected.


[]

In [13]:
%%sql
DROP TABLE IF EXISTS dimLocation;

CREATE TABLE dimLocation(
    Location VarChar,
    Primary_Instructor TEXT NOT NULL,
    title TEXT,
    term VarChar,
    act INT NOT NULL,
    cap INT NOT NULL,
    rem INT NOT NULL
);

INSERT INTO dimLocation(Location, Primary_Instructor, title, term, act, cap, rem)
SELECT DISTINCT Location, Primary_Instructor, title, term, act, cap, rem
FROM courses_final;

 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
13251 rows affected.


[]

In [14]:
%%sql

DROP TABLE IF EXISTS dimDate;

CREATE TABLE dimDate(
    term VarChar PRIMARY KEY,
    StartDate DATE NOT NULL,
    EndDate DATE NOT NULL    
);

INSERT INTO dimDate(term, StartDate, EndDate)
select term, StartDate, EndDate
FROM terms;

 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
16 rows affected.


[]

In [15]:
%%sql

DROP TABLE IF EXISTS factClass;

CREATE TABLE factClass ( 
    program_code VarChar,
    Term VarChar,
    Location VarChar,
    StartDate Date,
    EndDate Date,
    FOREIGN KEY (program_code) REFERENCES dimCourses(program_code),
    FOREIGN KEY (Term) REFERENCES dimTerm(Term),
    FOREIGN KEY (Location) REFERENCES dimLocation(Location),
    FOREIGN KEY (StartDate) REFERENCES dimDate(StartDate),  
    FOREIGN KEY (EndDate) REFERENCES dimDate(EndDate)
);

INSERT INTO factClass (program_code, Term, Location, StartDate, EndDate)
SELECT DISTINCT dimCourses.program_code, dimTerm.Term, dimLocation.Location, dimDate.StartDate, dimDate.EndDate
FROM dimCourses
LEFT JOIN dimTerm ON dimCourses.course_title = dimTerm.title
LEFT JOIN dimLocation ON dimTerm.Location = dimLocation.Location
LEFT JOIN dimDate ON dimLocation.Term = dimDate.Term;


 * sqlite:///CourseDataWarehouseFinal.db
Done.
Done.
56704 rows affected.


[]